In [2]:
import numpy as np
import pandas as pd
from decimal import Decimal
from plotly import express as px
import plotly.graph_objects as go
R = 0.1 # radius of the people
U = 0.2 # veloctiy of the field
PPM = 40000 # Expiratory carbon dioxide concentration
C_env = 440

# 最小值截断
def num_clip(values:list|float =[1e-15,1e15],min:float|None = 1e-10,max:float|None = None):
    if type(values) != list:
        values = [values]
    for i in range(len(values)):
        temp_value = np.clip(abs(values[i]),a_min=min,a_max=max)
        if temp_value == min:
            values[i] = 0
        elif temp_value == max:
            values[i] = np.inf
    return values

def positon_clac(vec=[0,0.55],R:float = 0.1,clip = 1e-15) -> list[float,float]:
    R = R
    try:
        theta = np.arctan(vec[1]/vec[0])
    except ZeroDivisionError:
        theta = np.arctan(np.inf)
    x = abs(R*np.cos(theta))
    y = abs(R*np.sin(theta))
    if vec[0] < 0:
        x = -x
    if vec[1] < 0:
        y = -y
    result = num_clip([x,y],min=clip)
    return result

def distance(s0=0,u_x=0,u_y=0,delta_t=0):
    result = s0+np.sqrt(u_x**2+u_y**2)*delta_t
    return result

def U_stockes(x,y,U = 0.2,R = 0.1):
    a = np.sqrt((x-(-R))**2+(y-0)**2)
    b = np.sqrt((x-0)**2+(y-0)**2) # distance from the center of the people to the point == r
    c = np.sqrt((-R-0)**2+(0-0)**2)
    d = round((b**2+c**2-a**2)/(2*b*c),12)
    rad_theta = np.arccos(d)
    u_r = -U*np.cos(rad_theta)*(1-3*R/(2*b)+R**3/(2*b**3))
    u_theta = U*np.sin(rad_theta)*(1-3*R/(4*b)-R**3/(4*b**3))
    u_x = round(u_r*(-np.cos(rad_theta))+u_theta*(np.sin(rad_theta)),10)
    u_y = round(u_r*(np.sin(rad_theta))+u_theta*(np.cos(rad_theta)),10)
    deg_theta = np.rad2deg(rad_theta).round(10)
    return[u_x,u_y,deg_theta,u_r,u_theta]

def dim(s):
    # s 为累计距离，单位m
    alpha = 0.076
    d0 = 0.02
    d = d0*6.8*(alpha*s/d0+1/6.8) # 0.147 = 1/6.8 使之满足s = 0时d = d0
    return d

def sim_jet(Total_time = 10,delta_t = 0.005,k = 0.5,U = 0.2,U_origin = [0,0.55]):
    x = positon_clac(U_origin)[0]
    y = positon_clac(U_origin)[1] # initial people position
    u_p_x = U_origin[0] # initial people velocity
    u_p_y = U_origin[1] # initial people velocity
    s = 0 # initial distance
    Q = np.sqrt(u_p_x**2+u_p_y**2)*np.pi*dim(s)**2/4 # initial flow rate
    res = [[x,y,u_p_x,u_p_y,s,dim(s),Q,PPM]]
    delta_t = delta_t # time step unit: 1/s
    for i in range(int(Total_time/delta_t)):
        alpha = 0.076
        delta_x = res[i][2]*delta_t
        delta_y = res[i][3]*delta_t
        x += delta_x
        y += delta_y
        s = distance(res[i][4],res[i][2],res[i][3],delta_t)
        U_stockes_origin = U_stockes(res[i][0],res[i][1],U=U)
        U_stockes_next = U_stockes(x,y,U=U)
        u_real = np.sqrt(res[i][2]**2+res[i][3]**2)
        delta_u_real_x = -(0.48*alpha/dim(s)*(res[i][2]-U_stockes_next[0])*abs(u_real)*delta_t*k/(0.147)**2)+U_stockes_next[0]-U_stockes_origin[0]
        delta_u_real_y = -(0.48*alpha/dim(s)*(res[i][3]-U_stockes_next[1])*abs(u_real)*delta_t*k/(0.147)**2)+U_stockes_next[1]-U_stockes_origin[1]
        u_real_x = res[i][2]+delta_u_real_x
        u_real_y = res[i][3]+delta_u_real_y
        Q2 = np.sqrt(u_real_x**2+u_real_y**2)*np.pi*dim(s)**2/4
        ppm = (PPM*Q+(Q2-Q)*C_env)/Q2
        res.append([x,y,u_real_x,u_real_y,s,dim(s),Q2,ppm,delta_u_real_x,delta_u_real_y
                    ,U_stockes_origin,U_stockes_next])
    return res

def transform_pd(data,columns = ['x','y','u_real_x','u_real_y','s','d','Q','ppm','d_u_x','d_u_y','d_U_s_x','d_U_s_y']):
    df = pd.DataFrame(data,columns = columns)
    # y_value = df['y'].iloc[0]
    # x_value = df['x'].iloc[0]
    # df['y'] = df['y'] - y_value
    # df['x'] = df['x'] - x_value
    return(df)

In [2]:
# 多工况下，夹角90度，U = 0.2
TIME = 20
DELTA_T = 0.005
K = 0.5
U = [0.2,0.4,0.6]
EXHALE = [0.55,0.8,1.1]

total_U = U[0]
exhale = EXHALE[0]
result = transform_pd(sim_jet(Total_time=TIME,delta_t=DELTA_T,k=K,U=0.2,U_origin=[0,exhale]))
result.to_csv(f'../../data/{exhale}_{total_U}.csv',index=False)
result

,x,y,u_real_x,u_real_y,s,d,Q,ppm,d_u_x,d_u_y,d_U_s_x,d_U_s_y
0,0.000000,0.000000,0.000000,0.550000,0.000000,0.020000,0.000173,40000.000000,NaN,NaN,None,None
1,0.000000,0.002750,0.008781,0.490401,0.002750,0.021421,0.000177,39109.695186,0.008781,-5.959938e-02,"[-0.0, 0.0, 90.0, 2.7192621468937822e-33, -2.2...","[0.007922709, 0.0, 90.0, -1.3041032182317608e-..."
2,0.000044,0.005202,0.015852,0.445651,0.005202,0.022689,0.000180,38353.488426,0.007070,-4.474916e-02,"[0.007922709, 0.0, 90.0, -1.3041032182317608e-...","[0.0144736315, -5.7395e-06, 90.0239124533, 3.0..."
3,0.000123,0.007430,0.021757,0.410454,0.007432,0.023841,0.000183,37692.954011,0.005906,-3.519790e-02,"[0.0144736315, -5.7395e-06, 90.0239124533, 3.0...","[0.0200482205, -2.1377e-05, 90.0656868916, 1.6..."
4,0.000232,0.009483,0.026819,0.381831,0.009487,0.024903,0.000186,37103.650558,0.005061,-2.862218e-02,"[0.0200482205, -2.1377e-05, 90.0656868916, 1.6...","[0.0248913718, -4.81046e-05, 90.1213872159, 4...."
...,...,...,...,...,...,...,...,...,...,...,...,...
3996,3.360217,0.086053,0.191119,0.000109,3.413755,1.784229,0.477855,454.304507,0.000003,-2.184802e-08,"[0.1910993798, -0.0002459571, 176.8299206272, ...","[0.1911018942, -0.0002458187, 176.8308110668, ..."
3997,3.361173,0.086053,0.191122,0.000109,3.414711,1.784723,0.478126,454.296404,0.000003,-2.183340e-08,"[0.1911018942, -0.0002458187, 176.8308110668, ...","[0.1911044072, -0.0002456804, 176.8317010153, ..."
3998,3.362128,0.086054,0.191124,0.000109,3.415666,1.785216,0.478397,454.288308,0.000003,-2.181889e-08,"[0.1911044072, -0.0002456804, 176.8317010153, ...","[0.1911069188, -0.0002455422, 176.8325904742, ..."
3999,3.363084,0.086054,0.191127,0.000109,3.416622,1.785710,0.478668,454.280219,0.000003,-2.190453e-08,"[0.1911069188, -0.0002455422, 176.8325904742, ...","[0.191109429, -0.0002454042, 176.8334794421, 0..."


In [4]:
# 单工况下，多夹角，U = 0.2
TIME = 20
DELTA_T = 0.005
EXHALE = 0.55
K = 0.5
U = 0.6
theta = np.arange(15,180,15)

u_x = -EXHALE*np.cos(np.deg2rad(theta))
u_y = EXHALE*np.sin(np.deg2rad(theta))

result = pd.DataFrame()
for i in range(len(theta)):
    temp = transform_pd(sim_jet(Total_time=TIME,delta_t=DELTA_T,k=K,U=U,U_origin=[u_x[i],u_y[i]]))
    res = pd.concat([temp['x'],temp['y']],axis=1)
    res.columns = [f'x-{theta[i]}',f'y-{theta[i]}']
    result = pd.concat([result,res],axis=1)

result.to_csv(f'../../data/position_degree_{U}_{EXHALE}.csv',index=False)

# px.scatter(result,x='x',y='y',range_x=[-0.5,0.5],range_y=[-0.5,0.5],width=800,height=800)